# Loading library

In [2]:
import pandas as pd
import numpy as np
import warnings
# Suppress RuntimeWarning: divide by zero encountered in divide
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Thuật toán dùng trong bài


## 1. Xử lý đầu vào
1.1 Về ràng buộc đẳng thức, bất đẳng thức --> chuyển về dạng chuẩn  
- nếu <=: giữ nguyên  
- nếu >=: nhân với -1  
- nếu =: chuyển ràng buộc Ax = b thành 2 ràng buộc Ax <= b và -Ax <= -b

1.2 Về ràng buộc dấu của biến 
- x>=0: giữ nguyên
- x<=0: thay x=-x, ghi nhớ vị trí biến để kết luận x=-x
- x không có ràng buộc: tạo 2 biến +x, -x mới, ghi nhớ vị trí biến để kết luận x = (+x) - (-x)

1.3 Về hướng của hàm mục tiêu  
- Nếu optimize_direction là max thì đưa hàm mục tiêu f(x) thành -f(-x)
- Nếu optimize_direction là min thì giữ nguyên

## 2. Thuật toán đơn hình Bland
Thuật toán đơn hình Bland để giải quyết bài toán QHTT khi:
- Tồn tại ci < 0 trên hàm mục tiêu
- Với mọi bi >= 0 trên ràng buộc đẳng thức, bất đẳng thức

... ghi thuật toán ...

## 3. Thuật toán đơn hình hai pha
Thuật toán đơn hình hai pha để giải quyết bài toán QHTT khi:
- Tồn tại ci < 0 trên hàm mục tiêu
- Tồn tại bi < 0 trên ràng buộc đẳng thức, bất đẳng thức

Thuật toán:
3.1 Pha 1 
- Lập bài toán bổ trợ
+ Thêm x_0 vào từ vựng ban đầu 
+ Lập từ vựng xuất phát (C) với hàm mục tiêu chỉ gồm biến x_0
- Chọn biến vào ra
+ Chọn biến vào: x_0
+ Chọn biến ra: biến ứng với dòng bi âm nhất, ưu tiên dòng có hệ số nhỏ nhất
- Xoay từ vựng như đơn hình
- Từ từ vựng thứ 2, áp dụng cách xoay như đơn hình để tìm được từ vựng cuối pha 1
- Kiểm tra từ vựng cuối pha 1
+ Nếu hàm mục tiêu chỉ gồm biến x0 --> chuyển qua pha 2
+ Ngược lại --> kết luận bài toán vô nghiệm

3.2 Pha 2
- Từ từ vựng tối ưu của pha 1
+ Cho x_0 = 0, lấy các ràng buộc ở từ vựng này (*)
+ Dùng hàm mục tiêu gốc (P') kết hợp với ràng buộc (*) --> hàm mục tiêu mới
- Xây dựng từ vựng mới bằng hàm mục tiêu mới kết hợp với các ràng buộc khi cho x_0 = 0
- Tiến hành xoay đơn hình trên từ vựng mới
- Kết luận nghiệm

## 4. Thuật toán đơn hình đối ngẫu
Thuật toán đơn hình đối ngẫu để giải quyết bài toán QHTT khi: 
- Tồn tại bi < 0 trên ràng buộc đẳng thức, bất đẳng thức

4.1 Bước 1: Xây dựng từ vựng xuất phát như bài toán đơn hình 

4.2 Bước 2: Quy tắc chọn biến vào ra 
- Nếu vẫn còn tồn tại bi < 0
+ Biến ra (i_0): chọn wi là biến ra có bi âm nhất 
+ Biến vào (j_0): trên dòng i_0, xét từng min{c_j/a_i0j} với(c_j > 0 và a_i0j > 0)
+ Xoay
- Nếu bi >= 0: giải quyết bài toán theo phương pháp đơn hình gốc/Bland

- Nếu {c_j/a_i0j} với(c_j > 0 và a_i0j > 0) không tồn tại --> Phương pháp khác

## 5. Dùng kỹ thuật đối ngẫu để giải bài toán
- Bước 1: Tìm bài toán đối ngẫu (D) của bài toán (P)
- Bước 2: Giải bài toán (D) bằng thuật toán đơn hình Bland hoặc đơn hình hai pha
- Bước 3: Chuyển từ vựng tối ưu của bài toán (D) sang từ vựng tối ưu của bài toán (P)
- Bước 4: Kết luận nghiệm theo bài toán (P) ban đầu



# Code

In [53]:
def processing_input(A, B):
    """ 
    # goal
    - chuyển 2 dataframe lưu ràng buộc đẳng thức/bất đẳng thức và ràng buộc về dấu của biến trong bài toán quy hoạch tuyến tính cho trước thành những ma trận có thể tính toán, xử lý được

    # input
    - A (dataframe): lưu ràng buộc đẳng thức/bất đẳng thức
    - B (dataframe): ràng buộc về dấu của biến 

    # output
    - optimize_direction: hướng của hàm mục tiêu
    - norm_arr (np.array): ma trận của bài toán sau khi chuyển về dạng chuẩn (P')
    - tab_arr (np.array): ma trận tương ứng với dạng bảng của phương pháp đơn hình của norm_arr
    - rsby (np.array): ma trận lưu trữ cách biểu diễn nghiệm của (P) theo (P')
    """

    # 1. Kiểm tra hướng của hàm mục tiêu
    A = A.fillna(0)
    optimize_direction = ''
    
    if A.iloc[0,-1] == 'max':
        optimize_direction = 'max'
    else:
        optimize_direction = 'min'
    A.iloc[0,-1] = 0

    # đưa cột cuối cùng của A về dạng numerical
    A[A.shape[1]-1] = pd.to_numeric(A[A.shape[1]-1])
    
    # nếu optimize_direction là max thì đưa hàm mục tiêu f(x) thành -f(-x)
    if optimize_direction == 'max':
        A.iloc[0,:] = pd.to_numeric(A.iloc[0,:])
        A.iloc[0,:] = -A.iloc[0,:]

    # 2. Xử lý ràng buộc về đẳng thức và bất đẳng thức
    # norm_arr là ma trận mới sau khi chuẩn hoá ma trận A. Dùng để làm biến đầu vào trong hàm xoay Bland
    A.iloc[0] = A.iloc[0].fillna(0)
    norm_arr = A.drop(columns=A.columns[A.shape[1] -1 -1]).to_numpy()
    nrow, ncol = norm_arr.shape
    j = 1
    for i in range(1,nrow):

        if A.iloc[i, -2] == ">=":
            norm_arr[i] = -norm_arr[i]
        elif A.iloc[i, -2] == "=":
            norm_arr = np.insert(norm_arr,j+1,[-norm_arr[j,:]],axis=0)
            j += 1
        j += 1

    # 3. Xử lý ràng buộc về dấu của biến
    # rsby là hàm result by vars: dùng để kết luận biến cũ theo biến mới sau khi chạy xong thuật toán xoay Bland
    rsby = np.eye(B.shape[1])
    k = 0
    for j in range(B.shape[1]):
        if B.iloc[0, j] == "<=":
            norm_arr[:, k] = -norm_arr[:,k]
            rsby[:, k] = -rsby[:,k]
        elif B.iloc[0, j] == ">=":
            "nothing"
        else:
            new_col_norm_A = -norm_arr[:, k]
            norm_arr = np.insert(norm_arr, k+1, new_col_norm_A, axis = 1)
            new_col_rsby = -rsby[:, k]
            rsby = np.insert(rsby, k+1, new_col_rsby, axis = 1)
            k = k+1
        k = k+1

    # 4. đưa norm_arr về dạng bảng của phương pháp đơn hình tab_arr
    # Đưa về dạng ma trận để có thể giải bài toán x1|x2|...|xn|w1|w2|...|wn|bi 
    n_var = norm_arr.shape[1] - 1
    n_const = norm_arr.shape[0] -1
    arr = np.insert(np.zeros((1, n_const)), 1, np.eye(n_const), axis = 0)
    tab_arr = np.hstack((norm_arr[:, :n_var], arr, norm_arr[:, n_var:]))
    # cho tab_arr[1:,:-1] = -tab_arr[1:,:-1]
    tab_arr[1:,:-1] = -tab_arr[1:,:-1]
    

    return(norm_arr, tab_arr, rsby, optimize_direction)


def select_input_output_index(arr):
    """
    # goal 
    - tìm chỉ số dòng i, chỉ số cột j của biến cần xoay trong thuật toán Bland

    # input
    - arr (np.array): ma trận tương tự dạng bảng của phương pháp đơn hình

    # output
    - nếu i = None, tức là không có biến vào --> từ vựng tối ưu
    - nếu i != None, j = None, tức là có biến vào, không có biến ra --> bài toán không giới nội
    - nếu i != None, j != None, trả về chỉ số (i,j) cần tìm
    """
    # Find the index of the first negative element, or None if no negative elements exist.
    const_arr = arr[0,:-1]
    lst = const_arr[const_arr<0]
    ind_in = None
    ind_out = None
    if len(lst) != 0:
        ind_in = np.where(const_arr<0)[0][0]
    else:
        # Tu vung toi uu
        return('TVTU')
    
    # Find the index of the largest negative number 
    lst = arr[1:, -1]/[i if i<0 else np.nan for i in arr[1:, ind_in]]
    if len(lst[lst<=0]) == 0:
        ind_out = None
    else:
        ind_out = np.where(lst == max(lst[lst<=0]))[0][0] + 1

    # Bai toan Khong gioi noi
    if ind_in != None and ind_out == None:
        return('KGN') 
    
    # Tra ve vi tri cua bien can xoay
    return(ind_in, ind_out)
 
    
def find_eye_list(arr_col):
    """ 
    # goal
    - Tìm vị trí dòng của biến cơ sở ở từ vựng tối ưu. Dùng để kết luận nghiệm (xét ở trường hợp duy nhất nghiệm)
    - Ở từ vựng tối ưu, nếu x_i là biến cơ sở, thì cột x_i - arr_col là mảng chỉ có số 0 và -1. Và có duy nhất một số -1 
    
    # input
    - arr_col: ma trận cột của từ vựng tối ưu

    # output
    - vị trí dòng của biến cơ sở
    """
    # Finds the index of the unique element equal to -1 in a binary array.
    lst = arr_col.tolist()
    for element in lst:
        if element not in [0,-1]:
            return None
    if lst.count(-1) == 1:
        return lst.index(-1)

def rotate(arr, i, j):
    """
    # goal
    - sử dụng thuật toán bland để xoay ma trận arr thành một ma trận mới arr_new

    # input
    - arr: mảng cần xoay
    - i: vị trí hàng của biến cần xoay
    - j: vị trí cột của biến cần xoay

    # output: return về mảng mới vừa được xoay
    """
    nrow, ncol = arr.shape
    arr_new = arr.copy()
    arr_new[i] = arr_new[i]/(-arr[i,j])
    for row in range(nrow):
        if row != i:
            for col in range(ncol):
                if col == j:
                    arr_new[row, col] = 0
                else:
                    arr_new[row, col] = arr[row, j] * arr_new[i, col] + arr[row, col]
    # Do lỗi lưu số thực của numpy (ví dụ: thay vì lưu số a = 0, thì có thể numpy lưu bằng -5.555557e-17) nên nhóm em chọn cách xử lý là làm tròn tới số thập phân thứ 13
    arr_new = np.round(arr_new, decimals = 13)
    return arr_new

def bland_algoritm(tab_arr):
    """ 
    # goal: sử dụng thuật toán bland để giải quyết bài toán quy hoạch tuyến tính khi tồn tại ci<0 trên hàm mục tiêu và bi>=0 trên ràng buộc đẳng thức, bất đẳng thức sau khi chuẩn hoá

    # input
    - tab_arr (np.array): ma trận từ vựng được tạo thành bởi hàm mục tiêu; các ràng buộc đẳng thức, bất đẳng thức; các ràng buộc về dấu của biến đã được chuẩn hoá

    # output
    - df_result (list): một danh sách lưu các từ vựng/ các bước giải của thuật toán 2 pha
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN) 
    """
    # tạo danh sách df_result để lưu lại các từ vựng tại mỗi lần xoay
    df_result = list()

    # thêm từ vựng ban đầu vào danh sách
    cur_arr = tab_arr
    df_result.append(cur_arr)

    # tạo biến type_result lưu các trường hợp xảy ra khi kết luận nghiệm: 
    # DDN: duy nhất nghiệm, KGN: không giới nội, VN: vô nghiệm, VSN: vô số nghiệm
    type_result = None

    # Xoay bland cho đến khi tìm được từ vựng tối ưu
    in_out_ind = select_input_output_index(cur_arr)
    while in_out_ind != 'TVTU':
        if in_out_ind != "KGN":
            j, i = in_out_ind
            new_arr = rotate(cur_arr, i, j)
            df_result.append(new_arr)
        else:
            type_result = "KGN"
            break
        cur_arr = new_arr
        in_out_ind = select_input_output_index(cur_arr)
    
    type_result = select_input_output_index(df_result[-1])

    return(df_result, type_result)


def two_phase_algorithm(norm_arr, tab_arr):
    """ 
    # goal: sử dụng thuật toán 2 pha để giải quyết bài toán quy hoạch tuyến tính khi tồn tại ci<0 trên hàm mục tiêu và bi<0 trên ràng buộc đẳng thức, bất đẳng thức sau khi chuẩn hoá

    # input
    - norm_arr (np.array): ma trận từ vựng được tạo thành bởi hàm mục tiêu; các ràng buộc đẳng thức, bất đẳng thức; các ràng buộc về dấu của biến đã được chuẩn hoá
    - tab_arr (np.array): ma trận norm_arr sau khi được đưa về dạng bảng của phương pháp đơn hình

    # output
    - df_result (list): một danh sách lưu các từ vựng/ các bước giải của thuật toán 2 pha
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN) 
    """

    # 1. Pha 1
    """ 
    1.1 Lập bài toán bổ trợ  
    - Thêm biến x0 vào ràng buộc đẳng thức, bất đẳng thức. Lập từ vựng xuất phát C = x0  
    - Đưa về dạng ma trận để có thể giải, lưu vào biến tab_C1 (np.array)
    """
    tab_C1 = tab_arr.copy()
    tab_C1 = np.insert(tab_C1, 0, np.ones(tab_C1.shape[0]), axis = 1)
    tab_C1[0, 1:] = 0

    """ 
    1.2 Xoay từ vựng 
    - Từ vựng đầu tiên
    + Chọn biến vào: x0
    + Chọn biến ra: biến ở dòng ứng với bi âm nhất, ưu tiên dòng có hệ số nhỏ nhất  
    - Từ vựng thứ 2 trở đi: áp dụng cách xoay đơn hình --> từ vựng cuối pha 1
    """
    # chọn biến vào là x_0
    j = 0
    # chọn biến ra, do ở thuật toán 2 pha, luôn tồn tại bi<0 nên luôn tồn tại biến ra i
    bi = tab_C1[1:, -1]
    i = np.where(bi == np.min(bi[bi<0]))[0][0] + 1
    # tạo df_result để lưu các từ vựng 
    df_result = list()
    # xoay từ vựng
    cur_arr = tab_C1
    df_result.append(cur_arr)
    in_out_ind = (j,i)
    while in_out_ind != 'TVTU':
        if in_out_ind != "KGN":
            j, i = in_out_ind
            new_arr = rotate(cur_arr, i, j)
            df_result.append(new_arr)
        else:
            type_result = "KGN"
            break
        cur_arr = new_arr
        in_out_ind = select_input_output_index(cur_arr)
    

    """ 
    1.3 Kiểm tra từ vựng cuối pha 1 (từ vựng tối ưu của P1), xét hàm mục tiêu
    - Nếu hàm mục tiêu chỉ có biến x0 --> chuyển sang pha 2
    - Nếu hàm mục tiêu gồm biến x0 và những biến khác (hoặc không có x0) --> kết luận bài toán vô nghiệm --> dừng thuật toán. 
    """
    type_result = None
    # hàm mục tiêu của từ vựng cuối pha 1
    obj_func_last_C1 = df_result[-1][0,:-1]
    # Nếu hàm mục tiêu chỉ có biến x0 --> pha 2
    if obj_func_last_C1[0] == 1 and np.all(obj_func_last_C1[1:] == 0):
        # 2. Pha 2
        """ 
        2.1 Tạo từ vựng mới
        - Từ từ vựng tối ưu của pha 1 (P1) -> cho x_0 = 0, lấy các ràng buộc ở từ vựng này
        - Dùng hàm mục tiêu gốc (P) kết hợp với hàm mục tiêu (P1) -> hàm mục tiêu mới
        """
        # Xoá cột x_0
        temp_arr_last_C1 = df_result[-1].copy()
        temp_arr_last_C1 = np.delete(temp_arr_last_C1,0,1)
        # tìm số biến, số ràng buộc của bài toán gốc (P)
        n_const = temp_arr_last_C1.shape[0] - 1
        n_var = temp_arr_last_C1.shape[1]-temp_arr_last_C1.shape[0]
        # tạo một ma trận biểu diễn x1, x2,... từ hàm mục tiêu ban đầu
        obj_fun_z = norm_arr[0,:-1].reshape(1,-1)
        # tạo một ma trận biểu diễn x1, x2,... từ hàm mục tiêu theo x1, x2, ..., w1, w2, ... có được trong từ vựng cuối pha 1, với shape = temp_arr_last_C1.shape
        rsby_C1 = np.zeros((n_var, n_const + n_var + 1))

        for i in range(n_var):
            if find_eye_list(temp_arr_last_C1[:,i]) != None:
                j = find_eye_list(temp_arr_last_C1[:,i])
                rsby_C1[i] = temp_arr_last_C1[j]
                rsby_C1[i,i] = 0
            else:
                rsby_C1[i,i] = 1

        z = obj_fun_z@rsby_C1
        # kết hợp z với ràng buộc của từ vựng cuối pha 1 --> tạo thành từ vựng đầu pha 2 tab_C2
        tab_C2 = np.insert(z, 1, temp_arr_last_C1[1:], axis = 0)

        """ 
        2.2 Tiến hành xoay Bland trên từ vựng mới --> kết luận nghiệm
        """
        cur_arr = tab_C2
        df_result.append(cur_arr)
        # Xoay bland cho đến khi tìm được từ vựng tối ưu
        in_out_ind = select_input_output_index(cur_arr)
        while in_out_ind != 'TVTU':
            if in_out_ind != "KGN":
                j, i = in_out_ind
                new_arr = rotate(cur_arr, i, j)
                df_result.append(new_arr)
            else:
                type_result = "KGN"
                # break
                return(df_result, type_result)
            cur_arr = new_arr
            in_out_ind = select_input_output_index(cur_arr)

        type_result = select_input_output_index(df_result[-1])


    # Nếu hàm mục tiêu gồm biến x0 và những biến khác (hoặc không có x0) --> kết luận bài toán vô nghiệm --> dừng thuật toán. 
    else:
        type_result = "VN"

    return(df_result, type_result)



def select_in_out_ind_selfdual(tab_arr):
    """
    # goal:
    - tìm chỉ số dòng i, chỉ số cột j của ma trận cần xoay, áp dụng cho ma trận tồn tại bi < 0 

    # input
    - tab_arr(np.array): ma trận tương tự dạng bảng của phương pháp đơn hình

    # output: luôn tồn tại i 
    - Nếu i!= None, j != None --> trả về chỉ số (j,i) cần tìm
    - Nếu i != None, j == None --> return None

    # Quy tắc chọn biến vào ra
    - Biến ra (i_0): chọn wi là biến ra có bi âm nhất 
    - Biến vào (j_0): trên dòng i_0, xét từng min{c_j/a_i0j} với (c_j > 0 và a_i0j > 0)
    """

    ind_in = None
    ind_out = None

    # Chọn biến ra là biến có bi âm nhất 
    bi = tab_arr[1:, -1]
    bi = np.array([i if i < 0 else np.inf for i in bi])
    ind_out = np.where(bi == np.min(bi))[0][0] + 1

    # Chọn biến vào là biến có {c_j/a_i0j} nhỏ nhất với (c_j > 0 và a_i0j > 0)
    row_obj_func = tab_arr[0,:-1]
    row_ind_out = tab_arr[ind_out, :-1]
    ci_ai = np.array([row_ind_out[i]/row_obj_func[i] if (row_ind_out[i] > 0 and row_obj_func[i] > 0) else np.inf for i in range(len(row_obj_func))])
    if np.any(ci_ai != np.inf):
        ind_in = np.where(ci_ai == np.min(ci_ai))[0][0]
    else:
        # Nếu không tồn tại biến vào ind_in
        return None
    
    # Trả về chỉ số của biến vào, ra cần tìm
    return(ind_in, ind_out)


def self_dual_simplex_algoritm(tab_arr):
    """ 
    # goal
    - Sử dụng thuật toán đơn hình đối ngẫu để giải quyết bài toán quy hoạch tuyến tinh
    
    # input
    - tab_arr (np.array): ma trận từ vựng dạng bảng của phương pháp đơn hình

    # output
    - df_result (list): một danh sách lưu các từ vựng/ các bước giải của thuật toán 2 pha
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN) 

    """

    # tạo danh sách df_result để lưu lại các từ vựng tại mỗi lần xoay
    df_result = list()
    # tạo biến type_result lưu các trường hợp xảy ra khi kết luận nghiệm: 
    type_result = None
    # thêm từ vựng ban đầu vào danh sách
    cur_arr = tab_arr
    df_result.append(cur_arr)

    bi = cur_arr[1:, -1]
    # Nếu vẫn còn tồn tại bi < 0, chọn biến vào ra bằng hàm select_in_out_ind_selfdual --> xoay từ vựng
    while np.any(bi < 0):
        if select_in_out_ind_selfdual(cur_arr) != None:
            j, i = select_in_out_ind_selfdual(cur_arr)
            new_arr = rotate(cur_arr, i, j)
            df_result.append(new_arr)
            cur_arr = new_arr
            bi = cur_arr[1:, -1]
        else:
            # Do phần này thầy chưa dạy, nên đặt tạm type_result = "VN"
            type_result = "VN"
            return(df_result, type_result)

    # Nếu bi >= 0, chọn biến vào ra bằng hàm select_input_output_index --> xoay từ vựng như phương pháp đơn hình/bland
    in_out_ind = select_input_output_index(cur_arr)
    while in_out_ind != 'TVTU':
        if in_out_ind != "KGN":
            j, i = in_out_ind
            new_arr = rotate(cur_arr, i, j)
            df_result.append(new_arr)
        else:
            type_result = "KGN"
            break
        cur_arr = new_arr
        in_out_ind = select_input_output_index(cur_arr)
    
    type_result = select_input_output_index(df_result[-1])

    return(df_result, type_result)



def  to_self_dual(A, B):
    """ 
    # goal
    - Tìm bài toán đối ngẫu (D) của bài toán quy hoạch tuyến tính ban đầu (P)

    # input
    - A (dataframe): lưu các ràng buộc đẳng thức, bất đẳng thức của bài toán (P)
    - B (dataframe): lưu các ràng buộc về dấu của biến của bài toán (P)

    # output 
    - new_const (dataframe): lưu các ràng buộc đẳng thức, bất đẳng thức của bài toán (D)
    - new_vars (dataframe): lưu các ràng buộc về dấu của biến của bài toán (D)
    """

    A = A.copy()
    B = B.copy()
    A = A.fillna(0)
    # array lưu các ràng buộc về đẳng thức, bất đẳng thức của bài toán mới
    new_const = A.to_numpy().transpose()
    new_const[-1,1:] = pd.to_numeric(new_const[-1,1:]) 
    new_const = np.concatenate((new_const[-1,:].reshape(1,-1),new_const[:-1,:]), axis=0)
    new_const = np.concatenate((new_const[:,1:], new_const[:,0].reshape(-1,1)), axis=1)

    # array lưu các ràng buộc về dấu của biến trong bài toán mới
    new_vars_sign = new_const[-1,:-1]
    new_const = new_const[:-1]
    new_const_sign = B.to_numpy(dtype=object)[0]
    
    # set optimize_direction cho bài toán mới
    optimize_direction_old = A.iloc[0,-1]
    if optimize_direction_old == "min":
        new_const[0,-1] = "max"
    else:
        new_const[0,-1] = "min"

    # set lại dấu cho ràng buộc đẳng thức, bất đẳng thức trong bài toán mới
    if optimize_direction_old == "min":
        # chuyển dấu của biến -> dấu của ràng buộc
        for i in range(len(new_const_sign)):
            if new_const_sign[i] == ">=":
                new_const_sign[i] = "<="
            elif new_const_sign[i] == "<=":
                new_const_sign[i] = ">="
            else:
                new_const_sign[i] = "="

        # chuyển dấu của ràng buộc thành dấu của biến
        for i in range(len(new_vars_sign)):
            if new_vars_sign[i] == ">=":
                new_vars_sign[i] = ">="
            elif new_vars_sign[i] == "<=":
                new_vars_sign[i] = "<="
            else:
                new_vars_sign[i] = 0
    else:
        # chuyển dấu của biến -> dấu của ràng buộc
        for i in range(len(new_const_sign)):
            if new_const_sign[i] == ">=":
                new_const_sign[i] = ">="
            elif new_const_sign[i] == "<=":
                new_const_sign[i] = "<="
            else:
                # new_const_sign[i] = str(new_const_sign[i])
                new_const_sign[i] = "="

        # chuyển dấu của ràng buộc thành dấu của biến
        for i in range(len(new_vars_sign)):
            if new_vars_sign[i] == ">=":
                new_vars_sign[i] = "<="
            elif new_vars_sign[i] == "<=":
                new_vars_sign[i] = ">="
            else:
                new_vars_sign[i] = 0

    new_const_sign = np.insert(new_const_sign, 0, 0)
    new_const = np.insert(new_const, -1, new_const_sign.reshape(1,-1), axis = 1)
    new_const = pd.DataFrame(np.array(new_const), columns=None)
    new_vars = pd.DataFrame(np.array(new_vars_sign.reshape(1,-1)),columns=None)

    return(new_const, new_vars)



def backward_primal_tab(dual_arr):
    """ 
    # goal 
    Đưa từ vựng của bài toán đối ngẫu (D) về từ vựng của bài toán (P)

    # Input
    - dual_arr : ma trận từ vựng tối ưu của bài toán (D)

    # Output
    - primal_arr: ma trận từ vựng tối ưu của bài toán (P)

    """
    # Tính số lượng biến, số lượng ràng buộc trong mỗi bài toán
    nvar_dual = dual_arr.shape[1] - dual_arr.shape[0]
    nconst_dual = dual_arr.shape[0] - 1 

    nvar_primal = nconst_dual
    nconst_primal = nvar_dual

    # Sắp xếp lại dual_arr theo thứ tự của primal_arr
    tmp_arr = np.concatenate([dual_arr[:, nvar_dual:-1], dual_arr[:, : nvar_dual], dual_arr[:,-1].reshape(-1,1)], axis=1)

    eye_list_j = [i for i in range(tmp_arr.shape[1] - 1) if find_eye_list(tmp_arr[:, i])!= None]
    tmp_arr_shortcut = np.delete(tmp_arr, eye_list_j, axis = 1)
    tmp_arr_shortcut = np.concatenate([tmp_arr_shortcut[:,-1].reshape(-1,1), tmp_arr_shortcut[:,:-1]], axis = 1).T
    
    # Tạo ma trận primal_arr
    primal_arr = np.zeros((nconst_primal+1, nconst_primal + nvar_primal + 1))
    primal_arr[:,-1] = tmp_arr_shortcut[:,0]
    eye_list_i = [find_eye_list(tmp_arr[:, j]) for j in eye_list_j]
    for n in range(len(eye_list_j)):
        i = eye_list_i[n]
        j = eye_list_j[n]
        primal_arr[:,j] = tmp_arr_shortcut[:,i]
        primal_arr[1:,j] = -primal_arr[1:,j]
    
    j = 1
    for i in range(primal_arr.shape[1] - 1):
        if np.all(primal_arr[:,i] == 0):
            primal_arr[j,i] = -1
            j += 1

    return(primal_arr)


def call_result(rsby, optimize_direction, df_result, type_result):
    """ 
    # goal
    - in kết quả của bài toán quy hoạch tuyến tính

    # input
    - rsby: ma trận biểu diễn của 1 biến
    - optimize_direction: biến lưu hướng tối ưu của hàm mục tiêu (max or min)
    - df_result: list lưu các bước giải của bài toán QHTT
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN)

    # output
    - show các bước giải, nếu lựa chọn show
    - show kết quả cuối cùng: 
        + btoan có nghiêm duy nhất: show nghiêm tối ưu, giá trị tối ưu
        + btoan vô nghiệm: nếu optimize_direction là max thì giá trị tối ưu là -inf, ngược lại
        + btoan không giới nội: nếu optimize_direction là max thì giá trị tối ưu là inf, ngược lại
        + btoan vô số nghiệm: cho các biến không cơ sở xuất hiện ở hàm mục tiêu bằng 0, viết biến cơ sở theo điều kiện các biến không cơ sở
    """
    
    # tách type_result "TVTU" thành "DDN" và "VSN"
    arr_tvtu = df_result[-1] # ma trận từ vựng tối ưu
    tvtu = arr_tvtu[0, :-1] # từ vựng tối ưu không lấy bi
    # if type_result == "TVTU":
    #     if np.count_nonzero(arr_tvtu[0,:-1]) <= 1: # sua < -> <=
    #         type_result = "VSN"
    #     else:
    #         type_result = "DNN"

    if type_result == "TVTU":
        # Tính số biến tự do
        nvar_free = rsby.shape[1] - rsby.shape[0]

        # Tính số biến bị khuyết ở hàm mục tiêu so với ràng buộc
        n_missing = 0
        n_const = arr_tvtu.shape[0] - 1
        for i in range(len(tvtu)):
            if tvtu[i] == 0 and np.any(arr_tvtu[1:,i] != 0):
                n_missing += 1
        n_missing = n_missing - n_const - nvar_free

        if n_missing >= 1:
            type_result = "VSN"
        else:
            type_result = "DNN"


    # Xét các trường hợp 
    """ 
    - x_P_hat là ma trận chứa nghiệm của các biến x1, ..., xn của (P')
    - x_P_hat là ma trận chứa nghiệm của các biến x1, ..., xn của (P) aka kết quả x cuối cùng
    - z là giá trị tối ưu của bài toán
    """
    x_P_hat = None
    x_P = None
    z = None
    # 1. Duy nhất nghiệm
    if type_result == "DNN":
        arr_tvtu = df_result[-1]
        result = np.zeros(arr_tvtu.shape[1])
        result[-1] = arr_tvtu[0, -1]
        for col in range(df_result[-1].shape[1] - 1):
            ind = find_eye_list(arr_tvtu[:,col])
            if ind != None:
                result[col] = arr_tvtu[ind, -1]
            else:
                result[col] = 0
        n_const = arr_tvtu.shape[0] - 1
        x_P_hat = result[:(len(result) - n_const - 1)]
        x_P = rsby @ np.array(x_P_hat)
        # x_P = np.array(x_P_hat)@rsby
        if optimize_direction == 'min':
            z = result[-1]
        else:
            z = -result[-1]
    
    # 2. Không giới nội
    elif type_result == "KGN":
        x_P = "Bai toan khong gioi noi"
        if optimize_direction == 'min':
            z = -np.inf
        else:
            z = np.inf

    # 3. Vô nghiệm
    elif type_result == "VN":
        x_P = "Bai toan vo nghiem"
        if optimize_direction == 'min':
            z = np.inf
        else:
            z = -np.inf

    # elif type_result == "VSN":
    #     """
    #     Xét trên từ vựng tối ưu ci>=0. Nếu tồn tại ci = 0 --> hàm mục tiêu bị khuyết so với các ràng buộc --> VSN
    #     - Cho biến cơ sở ở hàm mục tiêu bằng 0
    #     - Viết biến cơ sở theo biến khuyết
    #     - Tìm điều kiện, hệ điều kiện cho biến khuyết
    #     + Chỉ có 1 biến khuyết: tìm khoảng giới hạn của biến khuyết (gộp chung với có 2 biến khuyết)
    #     + Có từ 2 biến khuyết trở lên: viết hệ điều kiện 
    #     """
    #     # x_P lúc này dùng để lưu điều kiện cho biến x1, x2,..., w1, w2, ... được lưu với cú pháp 0:">=", 1:"="
    #     # z để lưu hệ điều kiện cho biến dưới dạng ma trận dòng x1, x2, ..., wn; cột x1, x2, ..., wn, bi
    #     x_P = tvtu
    #     tmp_arr_tvtu = arr_tvtu.copy()
    #     for i in range(len(x_P)):
    #         if x_P[i] != 0:
    #             x_P[i] = 1
    #             tmp_arr_tvtu[:, i] = 0
    #     z = np.zeros((tmp_arr_tvtu.shape[1] - 1, tmp_arr_tvtu.shape[1]))
    #     for i in range(z.shape[0]):
    #         j = find_eye_list(tmp_arr_tvtu[:,i])
    #         if j != None:
    #             z[i] = tmp_arr_tvtu[j]
    #             z[i,i] = 0

    elif type_result == "VSN":
        """
        Xét trên từ vựng tối ưu ci>=0. Nếu tồn tại ci = 0 --> hàm mục tiêu bị khuyết so với các ràng buộc --> VSN
        - Cho biến cơ sở ở hàm mục tiêu bằng 0
        - Viết biến cơ sở theo biến khuyết
        - Tìm điều kiện, hệ điều kiện cho biến khuyết
        + Chỉ có 1 biến khuyết: tìm khoảng giới hạn của biến khuyết (gộp chung với có 2 biến khuyết)
        + Có từ 2 biến khuyết trở lên: viết hệ điều kiện 
        """
        # x_P lưu điều kiện biến với 
        # x_P[0]: lưu điều kiện cho biến x1, x2,..., w1, w2, ... so với 0. Nếu "x >= 0" thì được lưu bằng 0, nếu "x = 0" thì được lưu bằng 1
        # X_P[1]: để lưu hệ điều kiện cho biến cơ sở theo biến không cơ sở. Được lưu dưới dạng ma trận dòng x1, x2, ..., wn; cột x1, x2, ..., wn, bi
        x_P = tvtu
        z = arr_tvtu[0,-1]
        tmp_arr_tvtu = arr_tvtu.copy()
        for i in range(len(x_P)):
            if x_P[i] != 0:
                x_P[i] = 1
                tmp_arr_tvtu[:, i] = 0
                
        cond_xP = np.zeros((tmp_arr_tvtu.shape[1] - 1, tmp_arr_tvtu.shape[1]))
        for i in range(cond_xP.shape[0]):
            j = find_eye_list(tmp_arr_tvtu[:,i])
            if j != None:
                cond_xP[i] = tmp_arr_tvtu[j]
                cond_xP[i,i] = 0
        x_P = [x_P, cond_xP]

        if optimize_direction == 'max':
            z = -z
    
    return(x_P, z, df_result)   



def solving_LP_problem(A, B):
    """
    # goal
    Hàm giải quyết bài toán quy hoạch tuyến tính (solving linear programming problem) bằng ba cách
    - Sử dụng thuật toán Bland
    - Sử dụng thuật toán đơn hình hai pha
    - Sử dụng thuật toán đơn hình đối ngẫu

    # input 
    - A (dataframe): lưu trữ các ràng buộc đẳng thức, bất đẳng thức của bài toán
    - B (dataframe): lưu trữ các ràng buộc về dấu của biến trong bài toán

    # output
    - Nếu bài toán được cho nằm trong 3 cách giải ứng với ba thuật toán trên thì in ra kết quả x, z và các bước giải
    - Ngược lại, trả về rỗng
    """

    nconst_A = A.shape[0] - 1
    # chuyển bài toán về dạng chuẩn 
    norm_arr, tab_arr, rsby, optimize_direction = processing_input(A, B)
    nconst = norm_arr.shape[0] - 1

    # arr_ci: biến lưu trữ các giá trị ci trên hàm mục tiêu
    arr_ci = tab_arr[0,:-1]
    # arr_bi: biến lưu trữ các giá trị bi trên từ vựng
    arr_bi = tab_arr[:,-1]

    # Nếu bài toán có 2 ràng buộc và hoặc là tất cả ci>=0 hoặc tất cả ci <= 0
    if nconst_A == 2 and (np.all(arr_ci>=0) or np.all(arr_ci<=0)):
    # if nconst_A == 2 and (np.all(np.array(A.iloc[0,:-2]) >= 0) or np.all(np.array(A.iloc[0,:-2]) >= 0)): 
        # Tạo bài toán đối ngẫu (P) của A, B
        C, D = to_self_dual(A,B)
        norm_arr1, tab_arr1, rsby1, optimize_direction1 = processing_input(C, D)
        
        arr_ci1 = tab_arr1[0,:-1]
        arr_bi1 = tab_arr1[:,-1]

        # Nếu bài toán (P) không nằm trong dạng các thuật giải đã học --> return
        if np.all(arr_ci1>=0) and np.all(arr_bi1>=0):
            df_result= [tab_arr]
            type_result = "TVTU"
            x_P, z, df_result = call_result(rsby, optimize_direction, df_result, type_result)
            return(x_P, z, df_result)
        # Nếu tồn tại ci < 0, với mọi bi >= 0 --> thuật toán đơn hình Bland
        if np.any(arr_ci1 < 0) and np.all(arr_bi1 >= 0):
            df_result, type_result = bland_algoritm(tab_arr1)
        # Nếu tồn tại bi < 0 --> thuật toán đơn hình đối ngẫu hoặc thuật toán đơn hình hai pha
        elif np.any(arr_bi1 < 0):
            df_result, type_result = two_phase_algorithm(norm_arr1, tab_arr1)
        
        # Nếu (D) không giới nội thì (P) vô nghiệm, ngược lại, nếu (D) vô nghiệm thì (P) không giới nội. 
        # Nếu bài toán nằm trong 2 trường hợp này thì không cần chuyển từ vựng tối ưu của (D) thành từ vựng tối ưu của (P)
        if type_result == "KGN":
            type_result = "VN"
        elif type_result == "VN":
            type_result = "KGN"
        else:
            # Chuyển ma trận từ vựng tối ưu của (D) thành ma trận từ vựng tối ưu của (P)
            primal_arr = backward_primal_tab(df_result[-1])
            df_result.append(primal_arr)

        x_P, z, df_result = call_result(rsby, optimize_direction1, df_result, type_result)
        return(x_P, z, df_result)

    # Nếu tồn tại ci < 0, với mọi bi >= 0 --> thuật toán đơn hình Bland
    if np.any(arr_ci < 0) and np.all(arr_bi >= 0):
        df_result, type_result = bland_algoritm(tab_arr)
    # Nếu tồn tại bi < 0 --> thuật toán đơn hình đối ngẫu hoặc thuật toán đơn hình hai pha
    if np.any(arr_bi < 0):
        df_result, type_result = two_phase_algorithm(norm_arr, tab_arr)
        


    # Trả kết quả của x, z và các bước giải quyết bài toán
    x_P, z, df_result = call_result(rsby, optimize_direction, df_result, type_result)
    return(x_P, z, df_result)


# Áp dụng 

In [113]:
A =  pd.read_csv("ex_const.csv", header=None)
B = pd.read_csv('ex_vars.csv', header=None)
if solving_LP_problem(A,B):
    x_P, z, df_result = solving_LP_problem(A,B)

    print(f"x = {x_P} \nz = {z} \n")
    for i in df_result:
        print(i,'\n')

x = [4. 0.] 
z = -8.0 

[[ 1.  0.  0.  0.  0.  0.]
 [ 1.  1. -1. -1. -0. -2.]
 [ 1.  2.  1. -0. -1. -1.]] 

[[ 0. -1.  1.  1.  0.  2.]
 [-1. -1.  1.  1.  0.  2.]
 [ 0.  1.  2.  1. -1.  1.]] 

[[ 1.  0.  0.  0.  0.  0.]
 [-1. -1.  1.  1.  0.  2.]
 [-1.  0.  3.  2. -1.  3.]] 

[[ 0.  2.  4.  0.  8.]
 [-1.  1.  1.  0.  2.]
 [ 0.  3.  2. -1.  3.]] 

[[ 0.  3.  2.  0.  8.]
 [-1. -2. -1.  0.  4.]
 [ 0. -3. -1. -1.  2.]] 



In [22]:
a = np.array([
    [0,4,-0,2,4],
    [0,5,1,2,1],
    [1,4,-0,1,2]
])
type_result = select_input_output_index(a)
type_result

'TVTU'

In [24]:
norm_arr, tab_arr, rsby, optimize_direction = processing_input(A, B)
np.all(a>=0)
norm_arr

array([[  2,   3,   1,   5,   1,   0],
       [  4,   2,   3,   1,   4,  50],
       [  3,   7,   1,   3,   2, 100]])

In [25]:
np.zeros(2)

array([0., 0.])